#**LAB-02 REDES CONVOLUCIONALES TRANSFER LEARNING**

##Nombre: Quispe Sucullani Jose David
##CU: 111-376
##Dirección de GitHub:

##CONTEXTO DEL LABORATORIO

Elaborar:

2.Realizar lo solicitado en el punto 1, pero utilizando un modelo preentrenado y aplicando técnicas de transfer learning y fine tunning. El modelo preentrenado no debe tener una antiguedad mayor a 5 años.

Se debe elaborar un informe en relación a los resultados que se obtengan en cada parte del trabajo.

El cuadernillo a presentar solo debe tener información que el estudiante considere necesaria.

Se debe subir el cuadernillo a ecampus y a su repositorio, para lo cual se debe incluir la dirección del mismo.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Comprobar si hay GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


#1. Preparación del Dataset

In [5]:
# Definir las transformaciones para las imágenes
transformaciones = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar las imágenes a 224x224 píxeles
    transforms.ToTensor(),  # Convertir las imágenes a tensores
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar usando los valores de los modelos preentrenados
])

In [6]:
# CARGAR LOS DATASET

# CARGAMOS LOS DATASET DE ENTRENAMIENTO Y PRUEBA

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
import os


# Cargar datasets
train_dir = '/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio_2/Dataset/train'
test_dir = '/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio_2/Dataset/test'

trainset = ImageFolder(train_dir, transform=transformaciones)
testset = ImageFolder(test_dir, transform=transformaciones)


print("Número de ejemplos en el conjunto de entrenamiento:", len(trainset))
print("Número de ejemplos en el conjunto de prueba:", len(testset))

Número de ejemplos en el conjunto de entrenamiento: 4015
Número de ejemplos en el conjunto de prueba: 4009


In [7]:
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=4)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [8]:
!pip install timm # install the missing module

In [9]:
import torch
import torch.nn as nn
import timm

class ModelCustom(torch.nn.Module):
    def __init__(self, n_outputs=5, pretrained=True, freeze=False):
        super().__init__()
        # Descargar el modelo ResNeSt pretrainado
        resnest = timm.create_model('resnest50d', pretrained=pretrained)

        # Replace the classifier layer
        num_ftrs = resnest.fc.in_features
        resnest.fc = torch.nn.Linear(num_ftrs, n_outputs)

        self.resnest = resnest

        if freeze:
            for param in self.resnest.parameters():
                param.requires_grad = False
            # Ensure the new classifier layer is trainable even if freeze is True
            for param in self.resnest.fc.parameters():
                param.requires_grad = True


    def forward(self, x):
        x = self.resnest(x)
        return x

    def unfreeze(self):
        for param in self.resnest.parameters():
            param.requires_grad = True

In [10]:
# Inicializar el modelo con ResNeSt
model = ModelCustom(n_outputs=5, pretrained=True, freeze=False)

# Ver la arquitectura del modelo
print(model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/110M [00:00<?, ?B/s]

ModelCustom(
  (resnest): ResNet(
    (conv1): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): ResNestBottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

## Cargar el modelo ResNeSt preentrenado

Aquí cargamos el modelo ResNet preentrenado y lo adaptamos para la clasificación de nuestras 5 clases de materiales organicos e inorganicos

In [11]:
import timm

# Cargar el modelo ResNeSt preentrenado
resnest = timm.create_model('resnest50d', pretrained=True)

# Ver la arquitectura del modelo
print(resnest)


ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResNestBottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (con

# Cargar el modelo preentrenado ResNeSt

In [12]:
import torch
import torch.nn as nn
import timm

# Configurar el dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Cargar el modelo preentrenado ResNeSt
modelo = timm.create_model('resnest50d', pretrained=True)

# Congelar las capas del modelo para evitar su entrenamiento inicial (Transfer Learning)
for param in modelo.parameters():
    param.requires_grad = False

# Reemplazar la última capa completamente conectada para que coincida con nuestro número de clases
num_ftrs = modelo.num_features  # Obtener el número de características de la última capa
modelo.fc = nn.Linear(num_ftrs, 5)  # 5 clases de materiales

# Mover el modelo al dispositivo adecuado
modelo = modelo.to(device)


##Definir la función de pérdida y el optimizador

Aplicaremos la técnica de Fine Tuning, por lo que seleccionamos un optimizador que ajustará solo los parámetros de la última capa.

In [13]:
# Definir la función de pérdida y el optimizador
criterio = nn.CrossEntropyLoss()
optimizador = optim.AdamW(modelo.parameters(), lr=0.001, weight_decay=0.01)


##Entrenamiento del modelo

El proceso de entrenamiento incluye la fase de Transfer Learning seguida de Fine Tuning. Primero, entrenaremos solo la última capa y luego desbloquearemos algunas capas adicionales para ajustar todo el modelo.

In [14]:
# Cargar el modelo preentrenado ResNeSt
modelo = timm.create_model('resnest50d', pretrained=True)

# Congelar las capas del modelo para evitar su entrenamiento inicial (Transfer Learning)
for param in modelo.parameters():
    param.requires_grad = False

# Reemplazar la última capa completamente conectada para que coincida con nuestro número de clases
num_ftrs = modelo.num_features  # Obtener el número de características de la última capa
modelo.fc = nn.Linear(num_ftrs, 5)  # 5 clases de materiales
# Mover el modelo al dispositivo adecuado
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelo = modelo.to(device)


In [15]:
# Función de entrenamiento
import os
def entrenar_modelo(modelo, criterion, optimizer, epochs=10, num_checkpoint=5):
    for epoch in range(epochs):
        modelo.train()
        running_loss = 0.0

        for inputs, labels in tqdm(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = modelo(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(trainloader.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Pérdida: {epoch_loss:.4f}")

        # Guardar el checkpoint cada num_checkpoint epochs
        if (epoch + 1) % num_checkpoint == 0:
            checkpoint_dir = '/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio_2'
            os.makedirs(checkpoint_dir, exist_ok=True) # Create directory if it doesn't exist
            checkpoint_path = f'{checkpoint_dir}/checkpoint_resnest_{epoch+1}.pth'
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': modelo.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': epoch_loss,
            }, checkpoint_path)
            print(f'Checkpoint saved to {checkpoint_path}')

In [ ]:
# Transfer Learning: Entrenar solo la última capa
entrenar_modelo(modelo, criterio, optimizador, epochs=5, num_checkpoint=2)

# Guardar el modelo completo después del entrenamiento de Transfer Learning
modelo_completo_path = '/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio_2/modelo_transfer_learning.pth'
torch.save(modelo.state_dict(), modelo_completo_path)
print(f'Modelo entrenado guardado en {modelo_completo_path}')

  0%|          | 0/63 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100%|██████████| 63/63 [1:40:54<00:00, 96.11s/it]


Epoch 1/5, Pérdida: 1.5682


 19%|█▉        | 12/63 [19:15<1:21:10, 95.51s/it]

##**FINE TUNING**

El fine tinig es para descongelas algunas capas para entrenar con todo el modelo.

In [ ]:
# Desbloquear todas las capas para Fine Tuning
for param in modelo.parameters():
    param.requires_grad = True

# Definir el optimizador para Fine Tuning
optimizador = optim.AdamW(modelo.parameters(), lr=0.0001, weight_decay=0.01)

# Reentrenar con Fine Tuning
entrenar_modelo(modelo, criterio, optimizador, epochs=10, num_checkpoint=5)

# Guardar el modelo completo después del Fine Tuning
modelo_completo_path = '/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio_2/modelo_fine_tuned.pth'
torch.save(modelo.state_dict(), modelo_completo_path)
print(f'Modelo fine-tuned guardado en {modelo_completo_path}')

##CARGAMOS LOS MODELOS ENTRENADOS.

In [ ]:
import torch
import torch.nn as nn
import copy

# Assuming 'modelo' is the original ResNet model architecture
modelo_transf_lear = copy.deepcopy(modelo)  # Create a new instance of the model
modelo_transf_lear.load_state_dict(torch.load('/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio_2/modelo_transfer_learning.pth'))
modelo_transf_lear.eval()  # Now you can call eval() on the new model instance

# Use modelo_transf_lear for further operations

##Evaluar el modelo

In [ ]:
def evaluar_modelo(modelo):
    modelo.eval()
    correctos = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = modelo(inputs)
            _, predicciones = torch.max(outputs, 1)
            total += labels.size(0)
            correctos += (predicciones == labels).sum().item()

    precision = 100 * correctos / total
    print(f"Precisión en el conjunto de prueba: {precision:.2f}%")

evaluar_modelo(modelo_transf_lear)


##PRUEBAS CON UNA IMAGEN FOTOGRÁFICA

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# Definir las transformaciones para las imágenes (debe coincidir con las del entrenamiento)
transform = transforms.Compose([
    transforms.Resize((50, 50)),  # Reducir el tamaño de la imagen a 50x50 píxeles
    transforms.ToTensor(),  # Convertir las imágenes a tensores
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar los tensores
])


In [ ]:
# Función para hacer predicciones
def predict_image(image_path):
    # Cargar la imagen
    image = Image.open(image_path)

    # Aplicar las transformaciones
    image_tensor = transform(image).unsqueeze(0)  # Añadir una dimensión para el batch

    # Mover el tensor de la imagen al mismo dispositivo que el modelo
    image_tensor = image_tensor.to(modelo_transf_lear.fc.weight.device)

    # Realizar la predicción
    with torch.no_grad():
        outputs = modelo_transf_lear(image_tensor)
        _, predicted = torch.max(outputs, 1)

    return predicted.item()

In [ ]:
# Ruta de la imagen a probar
# image_path = ''  # Cambia esto a la ruta de tu imagen
# image_path = ''
# image_path = ''
# image_path = ''
image_path = '/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio_2/Dataset/train/Cable-De-Red/imagen_6000.jpg'

# Obtener la clase predicha
predicted_class = predict_image(image_path)

# Mostrar la imagen y el resultado
image = Image.open(image_path)
plt.imshow(image)
plt.title(f'Predicted Class: {predicted_class}')
plt.axis('off')
plt.show()

In [ ]:
# OTRAS MANERA DE PREDECIR EL MODELO

# Cargar la imagen para prueba
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Añadir una dimensión para el batch
    return image

# Realizar una prueba con una imagen cargada
def predict(image_path):
    image = load_image(image_path)
    # Mover el tensor de la imagen al mismo dispositivo que el modelo
    image = image.to(device)
    with torch.no_grad():
        outputs = modelo_transf_lear(image)
        _, predicted = torch.max(outputs.data, 1)
        return predicted.item()

# Ejemplo de uso
image_path = '/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio_2/Dataset/train/Bolsas-Soda/imagen_2868.jpg'
predicted_class = predict(image_path)
print(f'Predicted class: {predicted_class}')

##PROBANDO OTRAS MANERA DE ENTRENAMIENTO DEL MODELO CON TRANSFER LEARNING

##Transfer Learning

In [ ]:
from tqdm import tqdm
import numpy as np
import torch.optim as optim
import torch.nn as nn
import os

def fit(model, dataloader_train, dataloader_test, epochs=5, lr=1e-2):
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader_train)
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")

        bar = tqdm(dataloader_test)
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")

        avg_train_loss = np.mean(train_loss)
        avg_train_acc = np.mean(train_acc)
        avg_val_loss = np.mean(val_loss)
        avg_val_acc = np.mean(val_acc)

        print(f"Epoch {epoch}/{epochs} loss {avg_train_loss:.5f} val_loss {avg_val_loss:.5f} acc {avg_train_acc:.5f} val_acc {avg_val_acc:.5f}")

        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(avg_train_acc)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(avg_val_acc)

    # Guardar el modelo completo al final del entrenamiento
    checkpoint_dir = '/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio_2'
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_path = os.path.join(checkpoint_dir, 'model_checkpoint.pth')
    torch.save({
        'epoch': epochs,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_val_loss,
    }, checkpoint_path)
    print(f'Modelo completo guardado en {checkpoint_path}')
    return history

In [ ]:
model_c = ModelCustom(pretrained=True, freeze=True)
fit(model_c, trainloader, testloader)

##Fine Runing

Todavía podemos mejorar un poco más si, además de utilizar los pesos descargados de Imagenet en resnet, entrenamos también la red completa.

In [ ]:
model_c = ModelCustom(pretrained=True, freeze=False)
fit(model_c, trainloader, testloader)

Es común entrenar primero el modelo sin entrenar la red pre-entrenada durante varias epochs y después seguir entrenando, pero permitiendo ahora la actualización de pesos también en la red pre-entrenada (usualmente con un learning rate más pequeño).

In [ ]:
model_o = ModelCustom(pretrained=True, freeze=True)
fit(model_o, trainloader, testloader)
model_o.unfreeze()
fit(model_o, trainloader, testloader, lr=1e-4)

Cargamos el modelo entrenado

In [ ]:
import torch
import torch.nn as nn
import copy

# Assuming 'modelo' is the original ResNet model architecture
model_transf_lear2 = copy.deepcopy(modelo)  # Create a new instance of the model
checkpoint = torch.load('/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio_2/modelo_transfer_learning.pth') # Load the checkpoint

# Print the keys of the checkpoint and the model's state_dict to compare
print("Checkpoint keys:", checkpoint['model_state_dict'].keys())
print("Model state_dict keys:", model_transf_lear2.state_dict().keys())


new_state_dict = {}
for key, value in checkpoint['model_state_dict'].items():
    # Example: Removing a 'resnet.' prefix from checkpoint keys if it exists
    new_key = key.replace('resnet.', '')
    new_state_dict[new_key] = value

# Load the modified state dictionary
model_transf_lear2.load_state_dict(new_state_dict, strict=False) # Set strict=False to ignore missing keys if needed
model_transf_lear2.eval()

# Use model_transf_lear2 for further operations

##Evaluación del modelo.

In [ ]:
def evaluar_modelo(modelo):
    modelo.eval()
    correctos = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = modelo(inputs)
            _, predicciones = torch.max(outputs, 1)
            total += labels.size(0)
            correctos += (predicciones == labels).sum().item()

    precision = 100 * correctos / total
    print(f"Precisión en el conjunto de prueba: {precision:.2f}%")

evaluar_modelo(model_transf_lear2)


In [ ]:
# Función para hacer predicciones
def predict_image2(image_path):
    # Cargar la imagen
    image = Image.open(image_path)

    # Aplicar las transformaciones
    image_tensor = transform(image).unsqueeze(0)  # Añadir una dimensión para el batch

    # Realizar la predicción
    with torch.no_grad():
        outputs = model_transf_lear2(image_tensor)
        _, predicted = torch.max(outputs, 1)

    return predicted.item()

##Hacemos pruebas con imagenes fotograficas del datasets creado con residuos organicos e inorganicos

In [ ]:
# Ruta de la imagen a probar
# image_path = '/content/drive/MyDrive/SIS420-IA2/Laboratorios/LAB-03/Images_prueba/acelga_08989.jpg'  # Cambia esto a la ruta de tu imagen
# image_path = '/content/drive/MyDrive/SIS420-IA2/Laboratorios/LAB-03/Images_prueba/cebolla_00386.jpg'
image_path = '/content/drive/MyDrive/Inteligencia-Artificial_2/Laboratorios/Laboratorio_2/Dataset/train/Cable-De-Red/imagen_6001.jpg'
# image_path = '/content/drive/MyDrive/SIS420-IA2/Laboratorios/LAB-03/Images_prueba/lechuga_00066.jpg'
# image_path = '/content/drive/MyDrive/SIS420-IA2/Laboratorios/LAB-03/Images_prueba/toronjil_00022.jpg'

# Obtener la clase predicha
predicted_class = predict_image2(image_path)

# Mostrar la imagen y el resultado
image = Image.open(image_path)
plt.imshow(image)
plt.title(f'Predicted Class: {predicted_class}')
plt.axis('off')
plt.show()

##DETALLES

- Transfer Learning: Se ha aplicado al congelar las capas iniciales del modelo preentrenado ResNet y solo entrenar la última capa completamente conectada.

- Fine Tuning: Se ha realizado desbloqueando las capas adicionales del modelo y ajustándolas mediante el optimizador.

- Regularización y Buenas Prácticas: Se han aplicado técnicas como la normalización de imágenes, optimización con SGD y el uso de DataLoader para una carga eficiente de datos.

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Training Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

# Assuming you have a 'history' object from your training
# plot_history(history) # Uncomment this line after you have trained the model and have a history object